# Laborato 7.2

### Organización de archivos para el correcto funcionamiento
```bash
docs/
    libro1.txt
    libro2.txt
    libro3.txt
    libro4.txt
    libro5.txt
    libro6.txt
df_total.csv
index_file.pkl
lab7.2.pdf
stoplist.txt
```

## P3- Matriz de similitudes
> Elabore una matriz de similitud de coseno entre los documentos de la colección "El Señor de los Anillos". Debe aplicar los pesos TF-IDF. 

### 1- Preprocesamiento

In [1]:
import nltk
import numpy as np
from nltk.stem.snowball import SnowballStemmer
nltk.download('punkt')
import math

[nltk_data] Downloading package punkt to /home/flowers/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
with open("stoplist.txt",encoding='latin-1') as file:
    stoplist = [line.strip() for line in file]
stoplist += [',','.','?','-',':',';','«','»','º','(',')']

In [3]:
import re
stemmer = SnowballStemmer('spanish')
def preprocesamiento(texto):
    words = []
    # tokenizar
    tokens = nltk.word_tokenize(texto.strip().lower())
    # filtrar stopwords
    for token in tokens:
        if token not in stoplist:
            words.append(token)
    # reducir palabras
    output = []
    for word in words:
        output.append(stemmer.stem(word))
    #output = list(set(output))
    return output

#preprocesamiento("docs/libro1.txt")
textos = ["libro1.txt","libro2.txt","libro3.txt","libro4.txt","libro5.txt","libro6.txt"]
textos_procesados = []
indice = {}
for file_name in textos:
  file = open('docs/'+file_name)
  texto = file.read().rstrip()
  texto = preprocesamiento(texto)  
  textos_procesados.append(texto)

### 2- Similitud de coseno

In [4]:
def compute_tfidf(collection):
  # calcular los pesos TF_IDF para cada documento de la coleccion

    N = len(collection)
    tf = [] # lista de diccionarios donde cada diccionario es un documento.
            #key: terminos y values: frecuencias de esos terminos en el documento.
    df = {} # Un solo diccionario donde keys: terminos 
            #y values: frecuencias en los que aparece en el documento.
    # calculando tf
    for doc in collection:
        term_freq = {}
        for term in doc:
            if term in term_freq:
                term_freq[term] += 1
            else:
                term_freq[term] = 1
        tf.append(term_freq)
        # calculando df
        for term in term_freq.keys():
            if term in df:
                df[term] += 1
            else:
                df[term] = 1

    # calculando tf-idf
    tf_idf = [] # Lista de diccionarios donde cada diccionario representa un documento.
                # key: terminos y value: peso TF-IDF de ese término en el documento.
    for list_dic in tf:
        dict_tfidf = {}
        for termino,frecuencia in list_dic.items():
            tf_value = math.log10(1 + frecuencia)
            idf_value = math.log10(N/df[termino])
            dict_tfidf[termino] = tf_value*idf_value
        tf_idf.append(dict_tfidf)
    return tf_idf

def cosine_sim(Q, Doc):
    # aplicar la similitud de coseno y construir la matriz
    # Si Q y Doc fueran vectores numericos
    #return np.dot(Q,doc) / (np.linalg.norm(Q) * np.linalg.norm(doc)) 
    # donde:
    # np.dot(Q,doc) producto punto entre los vectores Q y Doc.
    # np.linalg.norm la norma Euclidiana (longitud) del vector Q y doc. (pitagorazo)
    
    # Q,Doc ahora que son diccionarios 
    unique_terms = set(Q.keys()).union(set(Doc.keys()))

    # np.dot(Q,doc)
    dot_product = sum(Q.get(term, 0.0) * Doc.get(term, 0.0) for term in unique_terms)

    # np.linalg.norm(Q)
    norm_Q = math.sqrt(sum(value ** 2 for value in Q.values()))

    # np.linalg.norm(Doc)
    norm_Doc = math.sqrt(sum(value ** 2 for value in Doc.values()))

    # casos nulos
    if norm_Q == 0 or norm_Doc == 0:
        return 0.0
    return dot_product / (norm_Q * norm_Doc)

textos_tfidf = compute_tfidf(textos_procesados)
matriz = []
for doc1 in textos_tfidf:
  row = []
  for doc2 in textos_tfidf:  
    row.append(cosine_sim(doc1, doc2))
  matriz.append(row)

for row in matriz:
    print(row)

[0.9999999999999999, 0.05669275175863886, 0.04509387616430283, 0.04806343299920239, 0.0395922331872777, 0.0918719664473555]
[0.05669275175863886, 0.9999999999999991, 0.07179730207590511, 0.06183254395551965, 0.048636233918740786, 0.09700874413368801]
[0.04509387616430283, 0.07179730207590511, 1.0, 0.04414591567182407, 0.06708006727456846, 0.06932511845081603]
[0.04806343299920239, 0.06183254395551965, 0.04414591567182407, 0.9999999999999997, 0.06519740470538389, 0.08873197815455597]
[0.0395922331872777, 0.048636233918740786, 0.06708006727456846, 0.06519740470538389, 0.9999999999999996, 0.06443242375248782]
[0.0918719664473555, 0.09700874413368801, 0.06932511845081603, 0.08873197815455597, 0.06443242375248782, 1.0000000000000002]


## P4- Indice invertido con similitud de coseno

### 1- Estructura del índice invertido en Python:

In [5]:
""" 
index = {
w1 : [(doc1, tf_w1_doc1), (doc3, tf_w1_doc3),(doc4, tf_w1_doc4),(doc10, tf_w1_doc10)],
w2 : [(doc1, tf_w2_doc1 ), (doc2, tf_w2_doc2)],
w3 : [(doc2, tf_w3_doc2), (doc3, tf_w3_doc3),(doc7, tf_w3_doc7)],
}

idf = {
w1 : idf_w1,
w2 : idf_w2,
w3 : idf_w3,
}

length ={
doc1: norm_doc1,
doc2: norm_doc2,
doc3: norm_doc3,
...
}
"""

' \nindex = {\nw1 : [(doc1, tf_w1_doc1), (doc3, tf_w1_doc3),(doc4, tf_w1_doc4),(doc10, tf_w1_doc10)],\nw2 : [(doc1, tf_w2_doc1 ), (doc2, tf_w2_doc2)],\nw3 : [(doc2, tf_w3_doc2), (doc3, tf_w3_doc3),(doc7, tf_w3_doc7)],\n}\n\nidf = {\nw1 : idf_w1,\nw2 : idf_w2,\nw3 : idf_w3,\n}\n\nlength ={\ndoc1: norm_doc1,\ndoc2: norm_doc2,\ndoc3: norm_doc3,\n...\n}\n'

### 2- Algoritmo para construir el índice
#### Paso 1:	Implementar los métodos de la clase

In [6]:
import nltk
import numpy as np
import pandas as pd
from nltk.stem.snowball import SnowballStemmer
nltk.download('punkt')
import math
from collections import defaultdict
import pickle

stemmer = SnowballStemmer('spanish')

with open("stoplist.txt",encoding='latin-1') as file:
    stoplist = [line.strip() for line in file]
stoplist += [',','.','?','-',':',';','«','»','º','(',')']

[nltk_data] Downloading package punkt to /home/flowers/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
class InvertIndex: 

    def __init__(self, index_file):
        self.index_file = index_file
        self.index = defaultdict(list)
        self.idf = {}
        self.length = {}

    def preprocesamiento(self,texto):
        words = []
        # tokenizar
        tokens = nltk.word_tokenize(texto.strip().lower())
        # filtrar stopwords
        for token in tokens:
            if token not in stoplist:
                words.append(token)
        # reducir palabras
        output = []
        for word in words:
            output.append(stemmer.stem(word))
        #output = list(set(output))
        return output
    
    def load_index(self,index_file):
        with open(index_file,'rb') as file: 
            self.index,self.idf,self.length = pickle.load(file) # que es pickle? Sirve para serializar
    
    def save_index(self):
        with open(self.index_file,'wb') as file:
            pickle.dump((self.index,self.idf,self.length),file)
    
    def computeTF_IDF(self,processText):
        N = len(processText)
        tf = []
        df = {}
        # calculate tf
        for doc in processText:
            term_freq = {}
            for term in doc:
                if term in term_freq:
                    term_freq[term] += 1
                else:
                    term_freq[term] = 1
            tf.append(term_freq)
            # calculate df
            for term in term_freq.keys():
                if term in df:
                    df[term] += 1
                else:
                    df[term] = 1
        return tf,df
    
    def building(self, collection_text):
        # build the inverted index with the collection
        textos_procesados = []
        for row in collection_text: # imaginar que cada fila es un documento como en el ejercicio anterior
            textos_procesados.append(self.preprocesamiento(row))
        # compute the tf and df
        tf,df = self.computeTF_IDF(textos_procesados)
        
        # compute the idf
        N = len(textos_procesados)
        for term,freq in df.items():
            self.idf[term] = math.log10(N/freq) # dictionary idf 
                
        # compute the length (norm)
        for doc_id, term_freq in enumerate(tf):
            norm = 0
            for term, freq in term_freq.items():
                tf_idf =  math.log10(1+freq)*self.idf[term]
                norm += tf_idf**2
                self.index[term].append((doc_id,tf_idf)) # dictionary index
            self.length[doc_id] = math.sqrt(norm) # dictionary length
            
        # store in disk
        self.save_index()
    
    def getTerms(self, query):
        # extraer los terminos unicos
        return self.preprocesamiento(query)

    def getTf_idf(self,query):
        # calcular el tf-idf de la query
        tf_idf_query = {}
        # tf
        term_freq = defaultdict(int)
        for term in query:
            term_freq[term] += 1
        # tf-idf
        for term, freq in term_freq.items():
            if term in self.idf:
                tf_idf_query[term] = math.log10(1+freq)*self.idf[term]
            else:
                tf_idf_query[term] = 0
        return tf_idf_query

    def retrieval(self, query, k): # busqueda de los top k documentos
        self.load_index(self.index_file) # Leemos desde el disco a la RAM
        # diccionario para el score
        score = defaultdict(float) #{}
        # preprocesar la query: extraer los terminos unicos
        queryTerms = self.getTerms(query)
        # calcular el tf-idf del query
        tf_idf_query = self.getTf_idf(queryTerms)
        # aplicar similitud de coseno y guardarlo en el diccionario score
        query_norm = math.sqrt(sum(value**2 for value in tf_idf_query.values()))
        
        for term,weight in tf_idf_query.items():
            if term in self.index:
                for doc_id, tf_idf in self.index[term]:
                    score[doc_id] += tf_idf * weight
        for doc_id in score:
            score[doc_id] /= (self.length[doc_id]*query_norm)
        # ordenar el score de forma descendente
        result = sorted(score.items(), key= lambda tup: tup[1], reverse=True)
        # retornamos los k documentos mas relevantes (de mayor similitud al query)
        return result[:k]    

In [8]:
dataton = pd.read_csv('df_total.csv')
index_file = 'index_file.pkl'
invert_index = InvertIndex(index_file)
collection_text = list(dataton['news'])
invert_index.building(collection_text)

In [9]:
queries = [
    "La importancia del desarrollo en la sostenibilidad",
    "El pais de China y su cooperacion",
    "Economista importante de México",
    "Perú, delincuencia"
]
k = 5

In [10]:
def search_and_print_results(invert_index, queries, k):
    for i, query in enumerate(queries, start=1):
        print(f"------------------------------------")
        print(f"Results for Query {i}: {query}")
        result = invert_index.retrieval(query, k)
        for doc_id, score in result:
            print(f"Document ID: {doc_id}, Score: {score}")
        print(f"------------------------------------")
search_and_print_results(invert_index, queries, k)

------------------------------------
Results for Query 1: La importancia del desarrollo en la sostenibilidad
Document ID: 0, Score: 0.16247797364379105
Document ID: 917, Score: 0.14599615352698717
Document ID: 1076, Score: 0.13676608530187134
Document ID: 725, Score: 0.12402367954345637
Document ID: 601, Score: 0.11339766409182
------------------------------------
------------------------------------
Results for Query 2: El pais de China y su cooperacion
Document ID: 1, Score: 0.20882922202200407
Document ID: 15, Score: 0.14379238701980368
Document ID: 34, Score: 0.12640467194074168
Document ID: 157, Score: 0.11341349144334985
Document ID: 441, Score: 0.11081064693756089
------------------------------------
------------------------------------
Results for Query 3: Economista importante de México
Document ID: 313, Score: 0.19927392391962723
Document ID: 485, Score: 0.19927392391962723
Document ID: 648, Score: 0.17831018762694886
Document ID: 981, Score: 0.17831018762694886
Document ID: 

In [11]:
# Puedes corroborar manualmente aqui, poniendo el ID = indice
dataton = pd.read_csv('df_total.csv')
dataton['news'][754]

'Para julio de 2022 el Indicador de Confianza Empresarial ICE realizado por el Departamento Administrativo de Planeación Nacional de Estadísticas Dane para las empresas de comercio industria manufacturera servicios y construcción fue de 592 es decir 34 puntos porcentuales menos al que se registró en el mes anterior.Visto desde la parte de los sectores se conoció que las empresas dedicadas a servicios tuvieron una puntuación de 616 siendo la de mejor puntuación. Seguida están el sector de comercio con 59 puntos registrando junto con la anterior el valor más alto del indicador para julio de este año. A su vez el sector de industria manufacturera tuvo puntuación de 589 y construcción 507.Los resultados que más destacaron en la encuesta fue que en junio de 2022 441 de las empresas de los cuatro sectores estaban siendo gerenciadas o administradas por una mujer mientras que 559 por hombres. Por otra parte de estos cuatro sectores 975 de las empresas reportaron una operación normal para junio